## MCP Server

In [1]:
import os
os.makedirs("mcpcode",exist_ok=True)

In [ ]:
!pip install fastmcp --quiet

In [3]:
%%writefile mcpcode/mcpserver1.py 

from mcp.server.fastmcp import FastMCP
import wikipedia
import requests, json

mcp = FastMCP("myMCP")

@mcp.tool()
async def get_current_weather(city:str)->dict:
    """ can be used to get/fetch current weather information for a city name
    """
    api_key = "6a8b0ac166a37e2b7a38e64416b3c3fe"

    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"
    response = requests.get(url)
    response = response.content.decode()
    response = json.loads(response)
    output = {"City Name":city,"weather":response["weather"][0]['description'],
              "temperature":response['main']['temp'],
              "unit":"kelvin"}

    return output


@mcp.tool()
async def get_wikipedia_pages(query:str)->list:
    info = wikipedia.search(query)
    return info


if __name__=="__main__":
    mcp.run(transport="streamable-http")
    

Writing mcpcode/mcpserver1.py


In [4]:
%%writefile mcpcode/mcpserver2.py 

from mcp.server.fastmcp import FastMCP
import wikipedia
import requests, json

mcp = FastMCP("myMCP")

@mcp.tool()
def get_current_weather(city:str)->dict:
    """ can be used to get/fetch current weather information for a city name
    """
    api_key = "6a8b0ac166a37e2b7a38e64416b3c3fe"

    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"
    response = requests.get(url)
    response = response.content.decode()
    response = json.loads(response)
    output = {"City Name":city,"weather":response["weather"][0]['description'],
              "temperature":response['main']['temp'],
              "unit":"kelvin"}

    return output


@mcp.tool()
def get_wikipedia_pages(query:str)->list:
    info = wikipedia.search(query)
    return info


if __name__=="__main__":
    mcp.run(transport="stdio")
    

Overwriting mcpcode/mcpserver2.py


In [5]:
# run the MCP server
# python mcpcode/mcpserver1.py

### Implementing an Agent which uses tools from MCP server

In [13]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent

In [14]:
client = MultiServerMCPClient({"TredenceMCP":{"url":"http://127.0.0.1:8000/mcp",
                                              "transport":"streamable_http"}})

tools = await client.get_tools()
tools

[StructuredTool(name='get_current_weather', description=' can be used to get/fetch current weather information for a city name\n    ', args_schema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'title': 'get_current_weatherArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x74fceaad9300>),
 StructuredTool(name='get_wikipedia_pages', args_schema={'properties': {'query': {'title': 'Query', 'type': 'string'}}, 'required': ['query'], 'title': 'get_wikipedia_pagesArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x74fcea81dbc0>)]

In [15]:

agent = create_react_agent("azure_openai:gpt-4.1-mini",tools)


In [16]:
await agent.ainvoke({"messages":[{"role":"user","content":"what is weather in mumbai today?"}]})

{'messages': [HumanMessage(content='what is weather in mumbai today?', additional_kwargs={}, response_metadata={}, id='4711f2c1-abed-4f5e-8a60-945a2115b8cd'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_iBz2LeFUhT3szQMu9gN40WYL', 'function': {'arguments': '{"city":"Mumbai"}', 'name': 'get_current_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 77, 'total_tokens': 93, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_178c8d546f', 'id': 'chatcmpl-BeEYSqiJGIRxjslVwiqTm6ZBZlKsJ', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {

In [18]:
await agent.ainvoke({"messages":[{"role":"user","content":"tell me about wikipedia pages for Tata"}]})

{'messages': [HumanMessage(content='tell me about wikipedia pages for Tata', additional_kwargs={}, response_metadata={}, id='a3c3db04-03cb-4dcb-945e-078792fdd4d1'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_hbSecBSkerWmz1hpmdoRVFW8', 'function': {'arguments': '{"query":"Tata"}', 'name': 'get_wikipedia_pages'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 76, 'total_tokens': 94, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_178c8d546f', 'id': 'chatcmpl-BeEYqN3vy9dqjoQJmx5aGGg67xeKd', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_har